In [14]:
%pip install faiss-cpu sentence-transformers python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install PyMuPDF

1889.36s - thread._ident is None in _get_related_thread!


Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import fitz  # PyMuPDF for reading PDFs
import numpy as np
import requests
import faiss
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# Load .env variables (if you created one)
load_dotenv()

# Get your Groq API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

# Print to confirm
print("✅ Groq key loaded:", "✔️" if GROQ_API_KEY.startswith("gsk_") else "❌ (Replace your key manually)")

✅ Groq key loaded: ❌ (Replace your key manually)


In [5]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

def chunk_text(text, max_chars=500):
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

# Replace this with your actual PDF file name
pdf_path = "METASPOILT.pdf"
raw_text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(raw_text)

print(f"✅ Extracted {len(chunks)} chunks from the PDF.")
print("\n📌 First chunk:\n", chunks[0][:300], "...")

✅ Extracted 1277 chunks from the PDF.

📌 First chunk:
 David Kennedy, Jim O’Gorman, Devon Kearns, and Mati Aharoni
Foreword by HD Moore
Metasploit
The Penetration Tester’s Guide
METASPLOIT
METASPLOIT
The Penetration 
Tester’s Guide
by David Kennedy, 
Jim O’Gorman, Devon Kearns, 
and Mati Aharoni
San Francisco
METASPLOIT. Copyright © 2011 by David Kenned ...


In [7]:
# Load the embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Encode the chunks
embeddings = embedder.encode(chunks, convert_to_numpy=True)

# Show shape of embedding matrix
print("✅ Embeddings shape:", embeddings.shape)
print("📌 Example vector (first chunk):\n", embeddings[0][:5], "...")


d:\RAG Basics\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Embeddings shape: (1277, 384)
📌 Example vector (first chunk):
 [ 0.01040963  0.0466566  -0.0607655   0.04970527  0.01825979] ...


In [8]:
# Create a FAISS index
dimension = embeddings.shape[1]  # e.g., 768
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings)
faiss.write_index(index, "metasploit_index.faiss")
np.save("metasploit_chunks.npy", chunks)

print("✅ FAISS index created and populated.")
print("📌 Total vectors indexed:", index.ntotal)


✅ FAISS index created and populated.
📌 Total vectors indexed: 1277


In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load local FAISS index and original text chunks
index = faiss.read_index("metasploit_index.faiss")
chunks = np.load("metasploit_chunks.npy", allow_pickle=True)

# Load embedding model (same as used for chunk embeddings)
embedder = SentenceTransformer("all-MiniLM-L6-v2")


In [38]:
def retrieve_chunks(query: str, top_k=20):
    # Step 1: Embed the query using the same model
    query_embedding = embedder.encode([query], convert_to_numpy=True)

    # Step 2: Search in the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Step 3: Return the top-k matched text chunks
    return [chunks[i] for i in indices[0]]

In [37]:


def generate_answer(query: str, context_chunks):
    context = "\n".join([f"- {chunk}" for chunk in context_chunks])
    prompt = f"""You are a helpful assistant. Use only the given context to answer the question truthfully. Dont make up answers or hallucinate. If the answer is not in the context, say "I don't know".

Context:
{context}

Question: {query}
Answer:"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama3-8b-8192",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 8000
    }

    response = requests.post(GROQ_API_URL, headers=headers, json=payload)
    result = response.json()

    if "choices" in result:
        return result['choices'][0]['message']['content']
    else:
        raise Exception(f"Groq error: {result}")


In [39]:
query = "Give a summary about the joy of exploitation, more elaborate."
top_chunks = retrieve_chunks(query)
answer = generate_answer(query, top_chunks)

print("\n🧠 LLM Answer:")
print(answer)


🧠 LLM Answer:
Based on the provided context, the "Joy of Exploitation" chapter focuses on the art of exploiting vulnerabilities in a target system. The author emphasizes the importance of doing one's homework before launching an exploit, highlighting the need for precision rather than brute force. The chapter covers the basics of exploitation, including identifying potential vulnerabilities, launching an exploit, and compromising a target system.

The author also stresses the importance of post-exploitation, which involves identifying critical infrastructure, targeting information or data that the company values most, and demonstrating attacks that would have the greatest business impact. The chapter provides examples of how to use Metasploit, a penetration testing framework, to perform exploitation and post-exploitation tasks.

Throughout the chapter, the author provides guidance on how to avoid detection, use antivirus evasion techniques, and leverage vulnerability scanning technolo

In [11]:
#So Far we have done the RAG pipeline: Retrieval and Generation
# Next we are going to do it with langchain 
# S --- IGNORE --- 
print("Finished RAG pipeline with manual steps. Next, we will automate this using LangChain.")

Finished RAG pipeline with manual steps. Next, we will automate this using LangChain.
